%%

In [1]:
from consts import DATAROOT
import pandas as pd
import numpy as np
from pathlib import Path
import datetime

%%<br>
Reading data

%%

In [2]:
daily_stock = pd.read_csv(Path(DATAROOT, "stock_daily.csv"))

/Users/wayneliu/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


%%

In [3]:
monthly_stock_wide = pd.read_csv(Path(DATAROOT, 'signed_predictors_dl_wide.csv'))
monthly_stock_wide.head()

,permno,yyyymm,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AM,...,TrendFactor,UpRecomm,VarCF,VolMkt,VolSD,VolumeTrend,XFIN,zerotrade,zerotradeAlt1,zerotradeAlt12
0,10000,198601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,198602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.785175e-08,NaN
2,10000,198603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.023392e-07,NaN
3,10000,198604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.467463e-08,NaN
4,10000,198605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.649551e-08,NaN


%%

In [4]:
daily_stock.columns = (col.lower() for col in daily_stock.columns)
daily_stock['date'] = pd.to_datetime(daily_stock.date, format="%Y%m%d")
daily_stock.head()

,permno,date,siccd,nextdt,paydt,divamt,facpr,facshr,bidlo,askhi,prc,vol,bid,ask,shrout,cfacpr,cfacshr,openprc,retx
0,10078,1996-01-02,3570,NaN,NaN,NaN,NaN,NaN,43.625,46.00,44.750,4725864.0,44.625,44.750,183306.0,4.0,4.0,46.000,-0.019178
1,10078,1996-01-03,3570,NaN,NaN,NaN,NaN,NaN,40.125,43.75,41.250,8557526.0,41.125,41.250,183306.0,4.0,4.0,42.750,-0.078212
2,10078,1996-01-04,3570,NaN,NaN,NaN,NaN,NaN,37.000,42.00,40.500,15692762.0,40.500,40.625,183306.0,4.0,4.0,42.000,-0.018182
3,10078,1996-01-05,3570,NaN,NaN,NaN,NaN,NaN,38.500,41.75,40.500,9331474.0,40.375,40.500,183306.0,4.0,4.0,39.625,0.0
4,10078,1996-01-08,3570,NaN,NaN,NaN,NaN,NaN,39.875,41.50,40.125,1479269.0,40.000,40.125,183306.0,4.0,4.0,41.125,-0.009259


%%

In [5]:
reference = pd.read_csv(Path(DATAROOT, "CRSP_daily_stock_reference"))
reference['Variable Name'] = reference['Variable Name'].str.slice(0, -1)
reference = reference[reference['Variable Name'].isin(daily_stock.columns)]
reference['description_url'] = "https://wrds-www.wharton.upenn.edu/data-dictionary/form_metadata/crsp_a_stock_dsf_identifyinginformation/" + reference['Variable Name']
reference.head(2)

,Variable Name,Type,Description,description_url
10,siccd,double,SIC Code (siccd),https://wrds-www.wharton.upenn.edu/data-dictio...
20,prc,double,Price (prc),https://wrds-www.wharton.upenn.edu/data-dictio...


%%

In [6]:
sp500_op_ret = pd.read_csv(Path(DATAROOT, "sp500_op_ret.csv"))
sp500_op_ret['date'] = pd.to_datetime(sp500_op_ret['date'])
sp500_op_ret['exdate'] = pd.to_datetime(sp500_op_ret['exdate'])
sp500_op_ret.head()

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,optionid,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret
0,5015,1996-01-31,1996-07-20,C,20.0,4.625,5.000,15,25,0.380062,...,10980421,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367
1,5048,1996-01-31,1996-08-17,P,40.0,3.000,3.375,5,37,0.245716,...,11613022,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347
2,5049,1996-01-31,1996-05-18,C,65.0,5.875,6.375,4,420,0.167298,...,11618802,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721
3,5049,1996-01-31,1996-08-17,C,70.0,3.250,3.625,13,3806,0.149214,...,11770703,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186
4,5061,1996-01-31,1996-06-22,C,30.0,3.375,3.750,5,254,0.324523,...,10914516,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649


%%

In [7]:
mapping_table = pd.read_csv(Path(DATAROOT, "mapping_table.csv"))
mapping_table.head(2)
mapping_table['sdate'] = pd.to_datetime(mapping_table.sdate)
mapping_table['edate'] = pd.to_datetime(mapping_table.edate)

%% [markdown]<br>
# Add permno to monthly data

%%

In [8]:
(mapping_table.groupby('secid').count()[mapping_table.groupby('secid').permno.count() != 1])

,sdate,edate,permno
secid,,,
5007,2,2,2
5024,2,2,2
5047,2,2,2
5131,3,3,3
5139,2,2,2
...,...,...,...
213724,2,2,2
213726,2,2,2
213727,2,2,2


%%

In [9]:
(mapping_table.groupby('permno').count()[mapping_table.groupby('permno').secid.count() != 1])

,secid,sdate,edate
permno,,,
10113,3,3,3
10151,3,3,3
10157,2,2,2
10201,2,2,2
10220,2,2,2
...,...,...,...
93421,3,3,3
93424,3,3,3
93425,3,3,3


%%

In [10]:
mapping_table[mapping_table.permno == 10113]

,secid,sdate,edate,permno
19343,143581,2010-07-21,2020-12-31,10113
19345,143583,2016-09-01,2020-12-31,10113
19346,143584,2016-09-01,2020-12-31,10113


%%

In [11]:
mapping_table[mapping_table.secid == 5007]

,secid,sdate,edate,permno
5,5007,2001-04-09,2002-08-01,19692
6,5007,2012-04-12,2016-08-17,13343


%%

%%

In [12]:
def dates_overlaps(group):
    if group.shape[0] == 1:
        return False
    else:
        group.sort_values('sdate', inplace=True)
        return ((group.edate - group.edate.shift(1)) < datetime.timedelta(0)).any()

In [13]:
mapping_table.groupby('secid').apply(dates_overlaps)

secid
5001      False
5002      False
5004      False
5005      False
5006      False
          ...  
215155    False
215156    False
215158    False
215165    False
215166    False
Length: 27309, dtype: bool

%%

In [14]:
mapping_table.groupby('secid').apply(dates_overlaps)[mapping_table.groupby('secid').apply(dates_overlaps)]

secid
5505    True
9534    True
dtype: bool

%%

In [15]:
mapping_table[mapping_table.secid == 5505]

,secid,sdate,edate,permno
391,5505,1998-09-15,2020-12-31,79698
392,5505,1999-07-27,2005-07-06,87030


%% [markdown]<br>
## Summary on mapping table: if we eliminate the secids 5505 and 9534, each (secid, sdate, edate) triplet gives uniquely a permno, so we shall use this to link the data

%%

In [16]:
sp500_op_ret = sp500_op_ret[~sp500_op_ret.secid.isin([5505, 9534])]
sp500_op_ret = sp500_op_ret[sp500_op_ret.date <= mapping_table.edate.max()]

%%

In [17]:
def add_permno(group):
    secid = group.secid.iloc[0]
    mapping_group = MAPPING_GROUPED.get_group(secid)
    def find_permno(row):
        try:
            return mapping_group[(mapping_group.sdate <= row.date) & (mapping_group.edate >= row.date)].permno.iloc[0]
        except:
            print('occurs') 
            # sometimes it occurs that the date falls between one edate and one sdate in the mapping table. 
            # This event is rare (happens twice in the initial sample), so will just return nan and drop na later.
            return np.nan
    if mapping_group.shape[0] == 1:
        group['permno'] = mapping_group.permno.iloc[0]
    else:
        group['permno'] = group.apply(find_permno, axis=1)
    return group

In [18]:
MAPPING_GROUPED = mapping_table.groupby('secid')
grouped = sp500_op_ret.groupby('secid')
sp500_op_ret_w_permno = grouped.apply(add_permno).dropna(subset=['permno'])
sp500_op_ret_w_permno

occurs
occurs


,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret,permno
0,5015,1996-01-31,1996-07-20,C,20.0,4.625,5.000,15,25,0.380062,...,1,0,171,24.142672,24.000,24.0000,0.052253,4.8125,-0.025367,56266.0
1,5048,1996-01-31,1996-08-17,P,40.0,3.000,3.375,5,37,0.245716,...,1,0,199,39.344930,39.000,39.0000,0.051780,3.1875,0.002347,27051.0
2,5049,1996-01-31,1996-05-18,C,65.0,5.875,6.375,4,420,0.167298,...,1,0,108,70.168661,70.375,35.1875,0.053274,6.1250,-0.009721,19553.0
3,5049,1996-01-31,1996-08-17,C,70.0,3.250,3.625,13,3806,0.149214,...,1,0,199,70.393348,70.375,35.1875,0.051780,3.4375,0.001186,19553.0
4,5061,1996-01-31,1996-06-22,C,30.0,3.375,3.750,5,254,0.324523,...,1,0,143,32.001209,31.750,31.7500,0.052726,3.5625,-0.006649,10364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430491,214905,2020-12-31,2021-12-17,C,25.0,0.920,1.770,15,242,0.431994,...,1,0,351,18.770774,18.740,18.7400,0.001706,1.3450,-0.013468,20057.0
3430492,214905,2020-12-31,2021-12-17,C,27.5,0.800,1.170,1,83,0.438128,...,1,0,351,18.770774,18.740,18.7400,0.001706,0.9850,-0.007309,20057.0
3430493,214905,2020-12-31,2021-12-17,C,30.0,0.340,1.010,1,387,0.433132,...,1,0,351,18.770774,18.740,18.7400,0.001706,0.6750,-0.015556,20057.0
3430494,214905,2020-12-31,2021-12-17,C,40.0,0.180,0.430,16,472,0.485782,...,1,0,351,18.770774,18.740,18.7400,0.001706,0.3050,0.007335,20057.0


%%

In [19]:
sp500_op_ret_w_permno[sp500_op_ret_w_permno.option_ret.isna()]

,secid,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,cfadj,days_no_trading,days_to_exp,forwardprice,spotprice,adj_spot,ir_rate,mid_price,option_ret,permno
8053,5563,1996-03-29,1996-05-18,C,45.0,4.6250,4.875,1454,325,0.266162,...,1,0,50,49.364082,49.0000,49.0000,0.055139,4.75000,NaN,33785.0
8054,5563,1996-03-29,1996-07-20,C,35.0,14.3750,14.875,1,286,0.356107,...,1,0,113,49.751301,49.0000,49.0000,0.054911,14.62500,NaN,33785.0
8055,5563,1996-03-29,1996-07-20,C,45.0,5.2500,5.500,74,2151,0.226797,...,1,0,113,49.751301,49.0000,49.0000,0.054911,5.37500,NaN,33785.0
8056,5563,1996-03-29,1996-07-20,C,40.0,9.6250,10.125,150,420,0.291043,...,1,0,113,49.751301,49.0000,49.0000,0.054911,9.87500,NaN,33785.0
9583,5563,1996-03-29,1996-07-20,P,45.0,0.3125,0.500,31,343,0.181965,...,1,0,113,49.751301,49.0000,49.0000,0.054911,0.40625,NaN,33785.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945448,107289,2007-09-28,2007-11-17,P,25.0,0.2500,0.350,5,4716,0.423141,...,1,0,50,29.555588,29.3500,29.3500,0.052009,0.30000,NaN,11896.0
945449,107289,2007-09-28,2007-11-17,P,30.0,1.7500,1.850,254,12168,0.358659,...,1,0,50,29.555588,29.3500,29.3500,0.052009,1.80000,NaN,11896.0
1336605,104628,2010-06-30,2010-12-18,P,1.0,0.6600,0.740,5,1985,1.530905,...,1,0,171,0.343510,0.3425,0.3425,0.006329,0.70000,NaN,51043.0
1336606,104628,2010-06-30,2011-01-22,P,1.0,0.6900,0.750,30,24369,1.610486,...,1,0,206,0.343757,0.3425,0.3425,0.006530,0.72000,NaN,51043.0


%% [markdown]<br>
# Grouping daily data with month

%% [markdown]<br>
First, we check whether all monthly data are recorded on the last trading date of the natural month.

%%

In [20]:
dates = pd.Series(daily_stock.date.dt.strftime('%Y-%m-%d').unique())
end_of_trading_month = dates.groupby(dates.str.slice(0, 7)).max().values

%% [markdown]<br>
There is one wierd date.

%%

In [21]:
sp500_op_ret_w_permno[~sp500_op_ret_w_permno.date.dt.strftime('%Y-%m-%d').isin(end_of_trading_month)].date.unique()

array(['2017-09-29T00:00:00.000000000'], dtype='datetime64[ns]')

%%

In [22]:
daily_stock[daily_stock.date.dt.strftime('%Y-%m-%d') == '2017-09-30']

,permno,date,siccd,nextdt,paydt,divamt,facpr,facshr,bidlo,askhi,prc,vol,bid,ask,shrout,cfacpr,cfacshr,openprc,retx
922026,21178,2017-09-30,3761,NaN,20171229.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


%% [markdown]<br>
However, we checked that 2017-09-30 is Saturday, and since there is only one record in this regard, we simply drop this record.

%%

In [23]:
daily_stock = daily_stock[~(daily_stock.date.dt.strftime('%Y-%m-%d') == '2017-09-30')]

%% [markdown]<br>
Now everything's cool.

%%

In [24]:
dates = pd.Series(daily_stock.date.dt.strftime('%Y-%m-%d').unique())
end_of_trading_month = dates.groupby(dates.str.slice(0, 7)).max().values
sp500_op_ret_w_permno.date.dt.strftime('%Y-%m-%d').isin(end_of_trading_month).all()

True

%%

In [25]:
end_of_mon_dict = {date[0:7]: date for date in end_of_trading_month}

%%

In [26]:
def last_date(group):
    return pd.Series([group.date.max()] * group.shape[0], index=group.index)

In [27]:
daily_stock['month'] = daily_stock.groupby(daily_stock.date.dt.strftime('%Y-%m')).apply(last_date).droplevel(0)
daily_stock.sort_values('date', inplace=True)

%%

In [28]:
daily_stock['ind'] = daily_stock[~(daily_stock.siccd=='Z')].siccd.astype(str).str.slice(0,2).astype(int)


In [29]:
ind_info = daily_stock[['permno','month','ind']]
eom_ind = ind_info.groupby(['permno', 'month']).apply(lambda group: group.ind.iloc[-1]).dropna().astype(int).astype(str)

In [30]:
ind_dummies = pd.get_dummies(eom_ind)
ind_dummies.columns = [f'ind_{col}' for col in ind_dummies.columns]
ind_dummies = ind_dummies.reset_index(drop=False)
ind_dummies.columns = [(col if col!='month' else 'date') for col in ind_dummies.columns]
ind_dummies

,permno,date,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,ind_17,ind_20,...,ind_79,ind_80,ind_82,ind_83,ind_87,ind_89,ind_90,ind_95,ind_97,ind_99
0,10078,1996-01-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10078,1996-02-29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10078,1996-03-29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10078,1996-04-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10078,1996-05-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248097,93436,2021-08-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
248098,93436,2021-09-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
248099,93436,2021-10-29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
248100,93436,2021-11-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


%% [markdown]<br>
Now, when calculating the any monthly data, such as realized volatility, from daily data, we can simply do daily_stock.groupby(['permno', 'month']).apply(aggregation_func)

%% [markdown]<br>
# Aggregating calculations

%% [markdown]<br>
#### List of attributes to be calculated from the daily data<br>
    1.Realized volatility of stock price<br>
    2.Volume of stocks traded<br>
    3.Dollar volume of stocks traded

%%

In [31]:
reference

,Variable Name,Type,Description,description_url
10,siccd,double,SIC Code (siccd),https://wrds-www.wharton.upenn.edu/data-dictio...
20,prc,double,Price (prc),https://wrds-www.wharton.upenn.edu/data-dictio...
21,vol,double,Share Volume (vol),https://wrds-www.wharton.upenn.edu/data-dictio...
22,openprc,double,Open Price (openprc),https://wrds-www.wharton.upenn.edu/data-dictio...
23,askhi,double,Ask or High (askhi),https://wrds-www.wharton.upenn.edu/data-dictio...
24,bidlo,double,Bid or Low (bidlo),https://wrds-www.wharton.upenn.edu/data-dictio...
25,bid,double,Closing Bid (bid),https://wrds-www.wharton.upenn.edu/data-dictio...
26,ask,double,Closing Ask (ask),https://wrds-www.wharton.upenn.edu/data-dictio...
29,retx,double,Return without Dividends (retx),https://wrds-www.wharton.upenn.edu/data-dictio...
30,shrout,double,Number of Shares Outstanding (shrout),https://wrds-www.wharton.upenn.edu/data-dictio...


%%

In [32]:
attributes = []

%%

In [33]:
daily_stock['dollar_volu'] = daily_stock['vol'] * daily_stock['prc']

%%

In [34]:
grouped = daily_stock.groupby(['permno', 'month'])

%%

In [35]:
rvol = (grouped['prc'].std() * np.sqrt(12)).rename('rvol')
attributes += [rvol]

%%

In [36]:
share_volume = grouped['vol'].sum().rename('share_volume')
attributes += [share_volume]

%%

In [37]:
dollar_volume = grouped['dollar_volu'].sum().rename('dollar_volume')
attributes += [dollar_volume]

%%

In [38]:
attributes_from_daily_stock = pd.DataFrame(attributes).T.reset_index()
attributes_from_daily_stock

,permno,month,rvol,share_volume,dollar_volume
0,10078,1996-01-31,10.184240,171591183.0,7.309790e+09
1,10078,1996-02-29,10.481751,94697640.0,4.753652e+09
2,10078,1996-03-29,5.896730,121495097.0,5.597761e+09
3,10078,1996-04-30,12.058822,91977122.0,4.643751e+09
4,10078,1996-05-31,8.005711,86656717.0,5.079900e+09
...,...,...,...,...,...
248165,93436,2021-08-31,59.769163,381215577.0,2.686865e+11
248166,93436,2021-09-30,61.044351,388922784.0,2.940852e+11
248167,93436,2021-10-29,379.329488,526395492.0,4.830657e+11
248168,93436,2021-11-30,229.378048,645719701.0,7.233859e+11


%%

In [39]:
sp500_op_ret_w_permno = pd.merge(sp500_op_ret_w_permno, attributes_from_daily_stock, left_on=['permno', 'date'], right_on=['permno', 'month'], how='inner')

%% [markdown]<br>
# Revisit Daily option data<br>
From most of the characteristics in Bucket level, we need some volatility data on the options, so we dived back into the daily option data to hopefully make it possible to calculate more things.<br>
<br>
But failed...

%%<br>
daily_option = pd.read_csv(Path(DATAROOT, 'daily_option.csv'))<br>
# reading this takes \\ sadly forever... 

%%

%%

%%

%% [markdown]<br>
# Calculating Characteristics

%% [markdown]<br>
## Preliminary Work<br>
We first add some preliminary columns to the dataframe<br>
<br>
Most of the works here are contributed by Natasha

%%

In [40]:
sp500_op_ret_w_permno["yrs_to_exp"] = sp500_op_ret_w_permno["days_to_exp"] / 250
sp500_op_ret_w_permno["moneyness"] = sp500_op_ret_w_permno['strike_price'] / sp500_op_ret_w_permno['adj_spot']

# %%<br>
months = sp500_op_ret_w_permno['date'].sort_values(ascending=True).unique()<br>
months<br>
int_months = [int(month) for month in months]# np.searchsorted works wierdly on dt objects <br>
int_months

# %%<br>
# shift option return<br>
global I <br>
I = 0<br>
global L <br>
L = len(sp500_op_ret_w_permno.optionid.unique())<br>
# grouped = sp500_op_ret_w_permno.sort_values('date')[['optionid', 'date', 'option_ret']].groupby('optionid')<br>
grouped = sp500_op_ret_w_permno.sort_values('date').groupby('optionid')<br>
def get_all_months(group):<br>
    s_month, e_month = np.datetime64(group.date.iloc[0]), np.datetime64(group.date.iloc[-1])<br>
    return months[(months >= s_month) & (months <= e_month)]<br>
    # return months[np.searchsorted(months, s_month): np.searchsorted(months, e_month)+1]<br>
def get_prev_option_ret(group):<br>
    signal_df = pd.DataFrame(get_all_months(group), columns=['date'])<br>
    filled_group = pd.merge(group, signal_df, on='date', how='outer')<br>
    filled_group['prev_option_ret'] = filled_group.option_ret.shift(1)<br>
    global I<br>
    I+=1<br>
    print(I/L, end="\r")<br>
    return filled_group.dropna(subset=['optionid'])

# %%<br>
group = sp500_op_ret_w_permno.sort_values('date').groupby('optionid').get_group(10009003)

# %%<br>
sp500_op_ret_w_permno = grouped.apply(get_prev_option_ret).reset_index(drop=False)

%% [markdown]<br>
## Stock level

%% [markdown]<br>
Start to calculate the option characteristics

%%

In [41]:
sp500_op_ret_w_permno.columns

Index(['secid', 'date', 'exdate', 'cp_flag', 'strike_price', 'best_bid',
       'best_offer', 'volume', 'open_interest', 'impl_volatility', 'delta',
       'gamma', 'vega', 'theta', 'optionid', 'cfadj', 'days_no_trading',
       'days_to_exp', 'forwardprice', 'spotprice', 'adj_spot', 'ir_rate',
       'mid_price', 'option_ret', 'permno', 'month', 'rvol', 'share_volume',
       'dollar_volume', 'yrs_to_exp', 'moneyness'],
      dtype='object')

%%

In [42]:
stock_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date'])

%%<br>
fucntions calculating characteristics in part 1

toi

In [43]:
def toi(opt_data):
    sp500_op_ret = opt_data
    toi = sp500_op_ret.groupby(["secid", "date"])["open_interest"].sum()
    return toi
# toitest=toi(sp500_op_ret)

pcrratio

In [44]:
def pcratio(opt_data):
    sp500_op_ret = opt_data
    put_vol_permnth_underlying = sp500_op_ret.drop(sp500_op_ret[sp500_op_ret.cp_flag == 'C'].index)
    put_vol_permnth_underlying = put_vol_permnth_underlying.groupby(['secid', 'date'])["open_interest"].sum()
    put_vol_permnth_underlying.rename("oi_put", inplace=True)
    total_vol_permnth_underlying = sp500_op_ret.groupby(['secid', 'date'])["open_interest"].sum()
    total_vol_permnth_underlying.rename("oi_total", inplace=True)
    join = pd.concat([total_vol_permnth_underlying, put_vol_permnth_underlying], axis=1)
    join = join.fillna(0)
    ratio = join["oi_put"] / join["oi_total"]
    return ratio
# pcratiotest=pcratio(sp500_op_ret)

In [45]:
def vol(opt_data):
    sp500_op_ret = opt_data
    vol = sp500_op_ret.groupby(['secid', 'date'])["volume"].sum()
    return vol
# voltest=trading_vol(sp500_op_ret)

nopt

In [46]:
def nopt(opt_data):
    sp500_op_ret = opt_data
    nopt = sp500_op_ret.groupby(['secid', 'date'])["volume"].mean()
    return nopt
# nopttest=n_opt(sp500_op_ret)

dvol group by option id

In [47]:
def dvol(opt_data):
    sp500_op_ret = opt_data
    dvol = sp500_op_ret["volume"] * sp500_op_ret["mid_price"]
    sp500_op_ret["dvol_temp"] = dvol
    dvol = sp500_op_ret.groupby(['secid', 'date'])["dvol_temp"].sum()
    return dvol
# dvoltest=d_vol(sp500_op_ret)

ailliq

In [48]:
def ailliq(opt_data):
    sp500_op_ret = opt_data
    ailliq_a = abs(sp500_op_ret['prev_option_ret']) / sp500_op_ret['dvol_temp']
    sp500_op_ret["ailliq_a"] = ailliq_a                                      
    ailliq_a = sp500_op_ret.groupby(['secid', 'date'])["ailliq_a"].sum()                                          
    ailliq_b = sp500_op_ret.groupby(['secid', 'date'])["volume"].sum()
    ailliq = ailliq_a / ailliq_b
    return ailliq
# ailliqtest=a_illiq(sp500_op_ret)

pilliq

In [49]:
def pilliq(opt_data):
    sp500_op_ret = opt_data
    pilliq_a = abs(sp500_op_ret['prev_option_ret']) / sp500_op_ret['dvol_temp'] / sp500_op_ret['mid_price']
    sp500_op_ret["pilliq_a"] = pilliq_a                                      
    pilliq_a = sp500_op_ret.groupby(['secid', 'date'])["pilliq_a"].sum()                                          
    pilliq_b = sp500_op_ret.groupby(['secid', 'date'])["volume"].sum()
    pilliq = pilliq_a / pilliq_b
    return pilliq
# pilliqtest=p_illiq(sp500_op_ret)

##

In [50]:
def pcpv(opt_data):
    def get_PC_parity_vio(group):
        group.sort_values(['exdate', 'cp_flag'], ascending=False, inplace=True)
        s_grouped = group.groupby(['strike_price', 'exdate'])
        candi_key = []
        for key, s_group in s_grouped:  # record keys of such s_group that has length 2
            if s_group.shape[0] == 2:
                candi_key += [key]
        if not candi_key:
            return np.nan
        elif len(candi_key) == 1:
            pair = s_grouped.get_group(candi_key[0])
        else:
            current_spot = group.spotprice.iloc[0]
            moneyness = [abs(key[0] - current_spot) for keys in candi_key]
            pair_idx = candi_key[moneyness.index(min(moneyness))]
            pair = s_grouped.get_group(pair_idx)
        cal = pair.strike_price.iloc[0] * np.exp(-pair.ir_rate.iloc[0] * pair.days_to_exp.iloc[0] / 250) + pair.mid_price.iloc[1] - pair.mid_price.iloc[0]
        L = 100 * np.log(pair.spotprice.iloc[0] / cal)
        return L
    pcpv = opt_data.groupby(['secid', 'date']).apply(get_PC_parity_vio)
    return pcpv

pcpv=sp500_op_ret.groupby(['secid', 'date']).apply(get_PC_parity_vio)

In [51]:
def shrtfee(opt_data):
    def shrt_fee(group):
        group.sort_values(['exdate', 'cp_flag'], ascending=False, inplace=True)
        s_grouped = group.groupby(['strike_price', 'exdate'])
        candi_key = []
        for key, s_group in s_grouped:  # record keys of such s_group that has length 2
            if s_group.shape[0] == 2:
                candi_key += [key]
        if not candi_key:
            return np.nan
        elif len(candi_key) == 1:
            pair = s_grouped.get_group(candi_key[0])
        else:
            current_spot = group.spotprice.iloc[0]
            moneyness = [abs(key[0] - current_spot) for keys in candi_key]
            pair_idx = candi_key[moneyness.index(min(moneyness))]
            pair = s_grouped.get_group(pair_idx)
        cal = pair.adj_spot.iloc[0] - pair.strike_price.iloc[0] * np.exp(-pair.ir_rate.iloc[0] * pair.days_to_exp.iloc[0] / 250) + pair.mid_price.iloc[0] - pair.mid_price.iloc[1]
        L = (1 - cal / pair.adj_spot.iloc[0])**(1 / pair.strike_price.iloc[0])
        M = (1 - L) / (1 + pair.ir_rate.iloc[0])
        return M
    shrt_fee = opt_data.groupby(['secid', 'date']).apply(shrt_fee)
    return shrt_fee

iv_skew

In [52]:
def skewiv(opt_data):
    sp500_op_ret = opt_data
    put_vals = sp500_op_ret.drop(sp500_op_ret[sp500_op_ret.cp_flag == 'C'].index)
    call_vals = sp500_op_ret.drop(sp500_op_ret[sp500_op_ret.cp_flag == 'P'].index)
    temp_callvals = call_vals.groupby(['secid', 'date'])[["impl_volatility", "moneyness"]].mean()
    temp_putvals = put_vals.groupby(['secid', 'date'])[["impl_volatility", "moneyness"]].mean()
    otmput = temp_putvals[((temp_putvals["moneyness"] < 0.9))]
    atmcall = temp_callvals[
        (temp_callvals["moneyness"] >= 0.9) & (temp_callvals["moneyness"] <= 1.1)]
    atmcall = atmcall["impl_volatility"]
    otmput = otmput["impl_volatility"]
    atmcall.rename("implvolatmcall", inplace=True)
    otmput.rename("implvolotmput", inplace=True)
    skewiv_temp = pd.concat([otmput, atmcall], axis=1)
    skewiv = skewiv_temp["implvolotmput"] - skewiv_temp["implvolatmcall"]
    return skewiv
# skewivtest=iv_skew(sp500_op_ret)

atm_civpiv

In [53]:
def atm_civpiv(opt_data):
    sp500_op_ret = opt_data
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    atm_puts = atm_options.drop(atm_options[atm_options.cp_flag == 'C'].index)
    atm_calls = atm_options.drop(atm_options[atm_options.cp_flag == 'P'].index)
    atm_puts = atm_puts.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    atm_puts = atm_puts.rename(columns={"impl_volatility": "implvolput"})
    atm_calls = atm_calls.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    atm_calls = atm_calls.rename(columns={"impl_volatility": "implvolcall"})
    atm_civpiv = pd.concat([atm_calls, atm_puts], axis=1)
    atm_civpiv = atm_civpiv["implvolcall"] - atm_civpiv["implvolput"]
    return atm_civpiv
# atm_civpivtest= atm_civ_piv(sp500_op_ret)

atm_dcivpiv

In [54]:
def atm_dcivpiv(opt_data):
    sp500_op_ret = opt_data
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    atm_puts_dpiv = atm_options.drop(atm_options[atm_options.cp_flag == 'C'].index)
    dpiv = atm_puts_dpiv.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    dpiv['diffp'] = dpiv['impl_volatility'].diff()
    dpiv = dpiv['diffp']
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    atm_calls_cpiv = atm_options.drop(atm_options[atm_options.cp_flag == 'P'].index)
    cpiv = atm_calls_cpiv.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    cpiv['diffc'] = cpiv['impl_volatility'].diff()
    cpiv = cpiv['diffc']
    dcivpiv_temp = pd.concat([cpiv, dpiv], axis=1)
    atmdcivpiv = dcivpiv_temp["diffc"] - dcivpiv_temp["diffp"]
    return atmdcivpiv
# atm_dcivpiv=atm_dcivpiv_func(sp500_op_ret)

dpiv

In [55]:
def dpiv(opt_data):
    sp500_op_ret = opt_data
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    atm_puts_dpiv = atm_options.drop(atm_options[atm_options.cp_flag == 'C'].index)
    dpiv = atm_puts_dpiv.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    dpiv['diffp'] = dpiv['impl_volatility'].diff()
    dpiv = dpiv['diffp']
    return dpiv
# dpivtest=d_piv(sp500_op_ret)

dciv

In [56]:
def dciv(opt_data):
    sp500_op_ret = opt_data
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    atm_calls_cpiv = atm_options.drop(atm_options[atm_options.cp_flag == 'P'].index)
    cpiv = atm_calls_cpiv.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    cpiv['diffc'] = cpiv['impl_volatility'].diff()
    cpiv = cpiv['diffc']
    return cpiv
# dcivtest=d_civ(sp500_op_ret)

civpiv

In [57]:
def ntm_civpiv(opt_data):
    sp500_op_ret = opt_data
    ntm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.8) & (sp500_op_ret["moneyness"] <= 1.2)]
    ntm_puts = ntm_options.drop(ntm_options[ntm_options.cp_flag == 'C'].index)
    ntm_calls = ntm_options.drop(ntm_options[ntm_options.cp_flag == 'P'].index)
    ntm_puts = ntm_puts.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    ntm_puts = ntm_puts.rename(columns={"impl_volatility": "implvolput"})
    ntm_calls = ntm_calls.groupby(['secid', 'date'])[["impl_volatility"]].mean()
    ntm_calls = ntm_calls.rename(columns={"impl_volatility": "implvolcall"})
    ntm_civpiv = pd.concat([ntm_calls, ntm_puts], axis=1)
    ntm_civpiv = ntm_civpiv["implvolcall"] - ntm_civpiv["implvolput"]
    return ntm_civpiv
# civpivtest=ntm_civpiv(sp500_op_ret)

In [58]:
def ivd(opt_data):
    sp500_op_ret = opt_data
    impl_vol_sq = sp500_op_ret["impl_volatility"]**2
    sp500_op_ret["impl_vol_sq"] = impl_vol_sq
    sp500_op_ret.sort_values(["optionid", "days_to_exp"])
    temp_ivd = sp500_op_ret.groupby(['secid', 'date', 'days_to_exp'])[["impl_vol_sq"]].mean()
    temp_ivd = temp_ivd.reset_index(level=['days_to_exp', 'secid', 'date'])
    temp_ivd_diff = temp_ivd.groupby(['secid', 'date'])["impl_vol_sq"].diff()
    temp_ivd['impl_vol_sq_diff'] = temp_ivd_diff
    temp_ivd['impl_difft'] = temp_ivd['impl_vol_sq_diff'] * temp_ivd['days_to_exp']
    a = temp_ivd.groupby(['secid', 'date'])['impl_difft'].sum()
    b = temp_ivd.groupby(['secid', 'date'])['impl_vol_sq_diff'].sum()
    ivd = a / b
    return ivd
# ivdtest=iv_duration(sp500_op_ret)

iv_slope

In [59]:
def ivslope(opt_data):
    sp500_op_ret = opt_data
    atm_options = sp500_op_ret[(sp500_op_ret["moneyness"] >= 0.9) & (sp500_op_ret["moneyness"] <= 1.1)]
    short_term_options = atm_options[(sp500_op_ret["days_to_exp"] <= 90)]
    longterm_options = atm_options[(sp500_op_ret["days_to_exp"] > 90)]
    iv_longterm = longterm_options.groupby(['secid', 'date'])["impl_volatility"].mean()
    iv_shortterm = short_term_options.groupby(['secid', 'date'])["impl_volatility"].mean()
    iv_longterm.rename("ivlong", inplace=True)
    iv_shortterm.rename("ivshort", inplace=True)
    ivslope_temp = pd.concat([iv_longterm, iv_shortterm], axis=1)
    result = ivslope_temp["ivlong"] - ivslope_temp["ivshort"]
    return result
# ivslopetest=iv_slope(sp500_op_ret)

%%<br>
funcs_by_Natasha = [toi, pcratio, vol, nopt, dvol, ailliq, pilliq,<br>
                    pcpv, shrtfee, skewiv, atm_civpiv, atm_dcivpiv, <br>
                    dpiv, dciv, ntm_civpiv, ivd, ivslope]

In [60]:
funcs_by_Natasha = [toi, pcratio, vol, nopt, dvol, ailliq, pilliq,
                    pcpv, shrtfee, skewiv,
                    dpiv, dciv, ntm_civpiv, ivd, ivslope]  # deleted the two involving option returns

%%

In [61]:
results_Natasha = {}
problems = []
for func in funcs_by_Natasha:
    try:
        results_Natasha[func.__name__] = func(sp500_op_ret_w_permno)
    except Exception as e:
        print(f"""problem occured in {func.__name__}\n
              {e}""")
        problems.append(func.__name__)

problem occured in ailliq

              'prev_option_ret'
problem occured in pilliq

              'prev_option_ret'


/var/folders/t3/04qt42g934gglxrr79l_3lzr0000gn/T/ipykernel_16068/781060431.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  short_term_options = atm_options[(sp500_op_ret["days_to_exp"] <= 90)]
/var/folders/t3/04qt42g934gglxrr79l_3lzr0000gn/T/ipykernel_16068/781060431.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  longterm_options = atm_options[(sp500_op_ret["days_to_exp"] > 90)]


%%

In [62]:
for key in results_Natasha:
    results_Natasha[key].rename(key, inplace=True)

%%

In [63]:
STOCKLEVELCHARS = results_Natasha

%% [markdown]<br>
6. Stock vs. option volume (so)

%%

In [64]:
stock_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date'])

In [65]:
def get_so(group):
    agg_volu_option = group.volume.sum()
    return group.share_volume.iloc[0] / agg_volu_option

In [66]:
so = stock_level_grouped.apply(get_so).rename('so', inplace=True)
STOCKLEVELCHARS['so'] = so

%% [markdown]<br>
7. log of so

%%

In [67]:
def get_lso(so):
    return np.log(so)

In [68]:
lso = get_lso(so).rename('lso', inplace=True)
STOCKLEVELCHARS['lso'] = lso

/Users/wayneliu/miniforge3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


%% [markdown]<br>
8. dso

%%

In [69]:
stock_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date'])

In [70]:
def get_dso(group):
    agg_volu_option = (group.volume * group.mid_price).sum()
    return group.share_volume.iloc[0] / agg_volu_option

In [71]:
dso = stock_level_grouped.apply(get_dso).rename('dso', inplace=True)
STOCKLEVELCHARS['dso'] = dso

%% [markdown]<br>
9. ldso

%%

In [72]:
def get_ldso(dso):
    return np.log(dso)

In [73]:
ldso = get_ldso(dso).rename('ldso', inplace=True)
STOCKLEVELCHARS['ldso'] = ldso

%% [markdown]<br>
13. Proportional bid-ask spread (pba)

%%

In [74]:
stock_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date'])

In [75]:
def get_pba(group):
    return (group.volume * (group.best_offer - group.best_bid) / (0.5 * (group.best_offer - group.best_bid))).sum() / group.volume.sum()

In [76]:
pba = stock_level_grouped.apply(get_pba).rename('pba', inplace=True)
STOCKLEVELCHARS['pba'] = pba

%% [markdown]<br>
30. Weighted put-call spread (vs_level)

%%

In [77]:
stock_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date'])

In [78]:
def get_vs_level(group):
    group = group.sort_values('cp_flag', ascending=True)
    s_grouped = group.groupby(['strike_price', 'exdate'])
    candi_key = []
    for key, s_group in s_grouped:  # record keys of such s_group that has length 2
        if s_group.shape[0] == 2:
            candi_key += [key]
    if not candi_key:
        # print(f"{s_group.shape[0]} shape")
        return np.nan
    else:
        # print(1)
        weights = np.array([s_grouped.get_group(key).open_interest.sum() for key in candi_key])
        values = np.array([s_grouped.get_group(key).impl_volatility.iloc[0]
                           - s_grouped.get_group(key).impl_volatility.iloc[1] 
                           for key in candi_key])
        return sum(weights * values) / sum(weights)

In [79]:
vs_level = stock_level_grouped.apply(get_vs_level).rename('vs_level')
STOCKLEVELCHARS['vs_level'] = vs_level

%% [markdown]<br>
31. Change in Weighted put-call spread (vs_change)

%%

In [80]:
def vs_change(vs_level):
    grouped_by_secid = vs_level.sort_index(level=1).groupby(level=0)
    return grouped_by_secid.apply(lambda x: x - x.shift(1))

In [81]:
vs_change = vs_change(vs_level).rename('vs_change')
STOCKLEVELCHARS['vs_change'] = vs_change

%%

In [82]:
STOCKLEVELCHARS_df = pd.DataFrame(STOCKLEVELCHARS)

%% [markdown]<br>
# Bucket level<br>
<br>
What we can do here is indeed limited, since daily option data is unhandlable for us

%%

In [83]:
BUCKETLEVELCHARS = {}

%%

In [84]:
def tag_moneyness(row):
    if row.cp_flag == 'C':
        return 'OTM' if row.moneyness > 1.1 else ('ITM' if row.moneyness < 0.9 else 'ATM')
    else:
        return 'ITM' if row.moneyness > 1.1 else ('OTM' if row.moneyness < 0.9 else 'ATM')

In [85]:
sp500_op_ret_w_permno['moneyness'] = sp500_op_ret_w_permno.strike_price / sp500_op_ret_w_permno.adj_spot
sp500_op_ret_w_permno['moneyness_class'] = sp500_op_ret_w_permno.apply(tag_moneyness, axis=1)
sp500_op_ret_w_permno['maturity_class'] = sp500_op_ret_w_permno['days_to_exp'].map(lambda n: 'L' if n > 90 else 'S')
sp500_op_ret_w_permno['bucket_class'] = sp500_op_ret_w_permno['moneyness_class'] + ',' + sp500_op_ret_w_permno['maturity_class']

%% [markdown]<br>
14. Open Interest vs. stock volume (oistock)

%%

In [86]:
bucket_level_grouped = sp500_op_ret_w_permno.groupby(['secid', 'date', 'bucket_class'])

In [87]:
def get_oistock(group):
    stock_volu = group['share_volume'].iloc[0]
    return group.open_interest.sum() / stock_volu

In [88]:
oistock = bucket_level_grouped.apply(get_oistock).rename('iostock')
BUCKETLEVELCHARS['oistock'] = oistock

/var/folders/t3/04qt42g934gglxrr79l_3lzr0000gn/T/ipykernel_16068/3676158526.py:3: RuntimeWarning: divide by zero encountered in true_divide
  return group.open_interest.sum() / stock_volu


%% [markdown]<br>
15. Volume (bucket_vol)

%%

In [89]:
bucket_vol = bucket_level_grouped['volume'].sum().rename('bucket_vol')
BUCKETLEVELCHARS['bucket_vol'] = bucket_vol

%% [markdown]<br>
16. Dollor volume  (bucket_dvol)

%%

In [90]:
bucket_dvol = bucket_level_grouped['dollar_volume'].sum().rename('bucket_dvol')
BUCKETLEVELCHARS['bucket_dvol'] = bucket_dvol

%% [markdown]<br>
17. Relative Volume (bucket_vol_share)

%%

In [91]:
stock_vol = STOCKLEVELCHARS['vol']
bucket_vol_share = (bucket_vol / stock_vol).rename('bucket_vol_share')
BUCKETLEVELCHARS['bucket_vol_share'] = bucket_vol_share

%% [markdown]<br>
18. Turnover (turnover)

%%

In [92]:
turnover = (bucket_level_grouped['volume'].sum() / bucket_level_grouped['open_interest'].sum()).rename('turnover')
BUCKETLEVELCHARS['turnover'] = turnover

%%

In [93]:
BUCKETLEVELCHARS_df = pd.DataFrame(BUCKETLEVELCHARS)
BUCKETLEVELCHARS_df

oistock  bucket_vol   bucket_dvol  \
secid  date       bucket_class                                       
5015   1996-01-31 ATM,L         0.000020          16  4.815922e+08   
                  ATM,S         0.000003          11  9.631843e+08   
                  ITM,L         0.000001          15  4.815922e+08   
                  OTM,L         0.000003           1  4.815922e+08   
       1996-02-29 ATM,L         0.000036          50  3.404466e+08   
...                                  ...         ...           ...   
214905 2020-12-31 ATM,L         0.000147        1088  4.857248e+10   
                  ATM,S         0.000007        1090  1.942899e+10   
                  ITM,L         0.000011          35  1.942899e+10   
                  OTM,L         0.000070        1149  7.771597e+10   
                  OTM,S         0.000001         477  1.457174e+10   

                                bucket_vol_share  turnover  
secid  date       bucket_class                              
5015   1996-01-31 ATM,L                 0.372093  0.039702  
                  ATM,S                 0.255814  0.183333  
                  ITM,L                 0.348837  0.600000  
                  OTM,L                 0.023256  0.018182  
       1996-02-29 ATM,L                 0.218341  0.096154  
...                                          ...       ...  
214905 2020-12-31 ATM,L                 0.283407  0.026854  
                  ATM,S                 0.283928  0.564767  
                  ITM,L                 0.009117  0.011115  
                  OTM,L                 0.299297  0.059698  
                  OTM,S                 0.124251  1.358974  

[685643 rows x 5 columns]

%% [markdown]<br>
# Contract Level<br>
<br>
By Natasha

%%<br>
###############<br>
PART3 CHARACTERISTICS

call_indicator, put indicator

In [94]:
call_indicator = np.where((sp500_op_ret_w_permno["cp_flag"] == "C"), 1, 0)
sp500_op_ret_w_permno["C"] = call_indicator
put_indicator = np.where((sp500_op_ret_w_permno["cp_flag"] == "P"), 1, 0)
sp500_op_ret_w_permno["P"] = put_indicator

opt_spread

In [95]:
optspread = 2 * (sp500_op_ret_w_permno["best_offer"] - sp500_op_ret_w_permno["best_bid"]) / (sp500_op_ret_w_permno["best_offer"] + sp500_op_ret_w_permno["best_bid"])
sp500_op_ret_w_permno["optspread"] = optspread

embedlev

In [96]:
embedlev = sp500_op_ret_w_permno["spotprice"] / sp500_op_ret_w_permno["mid_price"] * abs(sp500_op_ret_w_permno["delta"])
sp500_op_ret_w_permno["embedlev"] = embedlev

volga calculation

In [97]:
d1 = np.log(sp500_op_ret_w_permno["spotprice"] / sp500_op_ret_w_permno["strike_price"]) + (sp500_op_ret_w_permno["ir_rate"] + 0.5 * sp500_op_ret_w_permno["impl_volatility"]) * sp500_op_ret_w_permno["yrs_to_exp"] / (sp500_op_ret_w_permno["impl_volatility"] * np.sqrt(sp500_op_ret_w_permno["yrs_to_exp"]))
d2 = d1 - sp500_op_ret_w_permno["impl_volatility"] * np.sqrt(sp500_op_ret_w_permno["yrs_to_exp"])
N = np.exp(-0.5 * d1**2) / 2 / np.pi
volga = np.sqrt(sp500_op_ret_w_permno["yrs_to_exp"]) * N * d1 * d2 / sp500_op_ret_w_permno["impl_volatility"]
sp500_op_ret_w_permno["volga"] = volga

%%

In [98]:
sp500_op_ret_w_permno['expiration_month'] = np.where((sp500_op_ret_w_permno["days_to_exp"] <= 21), 1, 0)
sp500_op_ret_w_permno['ttm'] = sp500_op_ret_w_permno["yrs_to_exp"]
sp500_op_ret_w_permno['iv'] = sp500_op_ret_w_permno['impl_volatility']
sp500_op_ret_w_permno['oi'] = sp500_op_ret_w_permno['open_interest']
sp500_op_ret_w_permno['doi'] = sp500_op_ret_w_permno['oi'] * sp500_op_ret_w_permno['mid_price']
sp500_op_ret_w_permno['mid'] = sp500_op_ret_w_permno['mid_price']

%%

In [99]:
Level_3_chars = ['C', 'P', 'expiration_month', 'ttm', 'moneyness', 'iv', 'delta',
                 'gamma', 'theta', 'vega', 'volga', 'embedlev', 'oi', 'doi',
                 'mid', 'optspread']  # , 'prev_option_ret']
intrinsics = ['secid', 'permno', 'date', 'exdate', 'optionid', 'bucket_class', 'option_ret']

%%

In [100]:
CONTRACTLEVELCHARS_df = sp500_op_ret_w_permno[intrinsics + Level_3_chars]

%% [markdown]<br>
# Merging results

%%

In [101]:
df0 = pd.merge(CONTRACTLEVELCHARS_df, ind_dummies, on=['permno', 'date'],how='inner')
df1 = pd.merge(df0, BUCKETLEVELCHARS_df.reset_index(), on=['secid', 'date', 'bucket_class'])
result = pd.merge(df1, STOCKLEVELCHARS_df.reset_index(), on=['secid', 'date'])

%%

In [102]:
result

,secid,permno,date,exdate,optionid,bucket_class,option_ret,C,P,expiration_month,...,ntm_civpiv,ivd,ivslope,so,lso,dso,ldso,pba,vs_level,vs_change
0,5015,56266.0,1996-01-31,1996-07-20,10980421,"ITM,L",-0.025367,1,0,0,...,0.045109,171.0,0.000491,4.758047e+05,13.072763,1.738007e+05,12.065664,2.0,NaN,NaN
1,5015,56266.0,1996-01-31,1996-07-20,10385280,"ATM,L",-0.003343,1,0,0,...,0.045109,171.0,0.000491,4.758047e+05,13.072763,1.738007e+05,12.065664,2.0,NaN,NaN
2,5015,56266.0,1996-01-31,1996-07-20,10440757,"OTM,L",-0.049677,1,0,0,...,0.045109,171.0,0.000491,4.758047e+05,13.072763,1.738007e+05,12.065664,2.0,NaN,NaN
3,5015,56266.0,1996-01-31,1996-04-20,10752997,"ATM,S",-0.012452,0,1,0,...,0.045109,171.0,0.000491,4.758047e+05,13.072763,1.738007e+05,12.065664,2.0,NaN,NaN
4,5015,56266.0,1996-01-31,1996-04-20,11214327,"ATM,S",-0.032655,0,1,0,...,0.045109,171.0,0.000491,4.758047e+05,13.072763,1.738007e+05,12.065664,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430485,213780,18724.0,2020-12-31,2021-02-19,137483966,"ATM,S",-0.064394,0,1,0,...,-0.052181,197.0,-0.027641,2.152519e+06,14.582149,2.988727e+06,14.910358,2.0,-0.032158,-0.015836
3430486,213780,18724.0,2020-12-31,2021-02-19,137483951,"OTM,S",0.027385,1,0,0,...,-0.052181,197.0,-0.027641,2.152519e+06,14.582149,2.988727e+06,14.910358,2.0,-0.032158,-0.015836
3430487,213780,18724.0,2020-12-31,2021-07-16,137032868,"ATM,L",0.017358,1,0,0,...,-0.052181,197.0,-0.027641,2.152519e+06,14.582149,2.988727e+06,14.910358,2.0,-0.032158,-0.015836
3430488,213780,18724.0,2020-12-31,2021-07-16,137032869,"ATM,L",-0.009610,1,0,0,...,-0.052181,197.0,-0.027641,2.152519e+06,14.582149,2.988727e+06,14.910358,2.0,-0.032158,-0.015836


%%

In [103]:
result.to_csv(Path(DATAROOT, 'option_characteristics.csv'))

%%

In [104]:
result['yyyymm'] = result.date.dt.strftime('%Y%m').astype(int)

%%

In [105]:
result['yyyymm'].iloc[1]

199601

%%

In [106]:
monthly_stock_wide.yyyymm.iloc[1]

198602

%%

In [107]:
all_chars_df = pd.merge(result, monthly_stock_wide, on=['permno', 'yyyymm'], how='inner')

%%

In [108]:
all_chars_df.to_csv(Path(DATAROOT, 'all_characteristics.csv'))

In [111]:
list(all_chars_df.columns)

['secid',
 'permno',
 'date',
 'exdate',
 'optionid',
 'bucket_class',
 'option_ret',
 'C',
 'P',
 'expiration_month',
 'ttm',
 'moneyness',
 'iv',
 'delta',
 'gamma',
 'theta',
 'vega',
 'volga',
 'embedlev',
 'oi',
 'doi',
 'mid',
 'optspread',
 'ind_10',
 'ind_12',
 'ind_13',
 'ind_14',
 'ind_15',
 'ind_16',
 'ind_17',
 'ind_20',
 'ind_21',
 'ind_22',
 'ind_23',
 'ind_24',
 'ind_25',
 'ind_26',
 'ind_27',
 'ind_28',
 'ind_29',
 'ind_30',
 'ind_31',
 'ind_32',
 'ind_33',
 'ind_34',
 'ind_35',
 'ind_36',
 'ind_37',
 'ind_38',
 'ind_39',
 'ind_40',
 'ind_41',
 'ind_42',
 'ind_44',
 'ind_45',
 'ind_46',
 'ind_47',
 'ind_48',
 'ind_49',
 'ind_50',
 'ind_51',
 'ind_52',
 'ind_53',
 'ind_54',
 'ind_55',
 'ind_56',
 'ind_57',
 'ind_58',
 'ind_59',
 'ind_60',
 'ind_61',
 'ind_62',
 'ind_63',
 'ind_64',
 'ind_65',
 'ind_67',
 'ind_70',
 'ind_72',
 'ind_73',
 'ind_75',
 'ind_76',
 'ind_78',
 'ind_79',
 'ind_80',
 'ind_82',
 'ind_83',
 'ind_87',
 'ind_89',
 'ind_90',
 'ind_95',
 'ind_97',
 'ind

%%<br>
all_chars_df